In [ ]:
%load_ext autoreload
%autoreload 2
import os
import sys
import panel as pn
import numpy as np
import pyvista as pv
pv.set_plot_theme("document")
import glob
from matplotlib.colors import ListedColormap
from omegaconf import OmegaConf
from torch_geometric.data import Data
import random
import torch

pn.extension('vtk')
os.system('/usr/bin/Xvfb :99 -screen 0 1024x768x24 &')
os.environ['DISPLAY'] = ':99'
os.environ['PYVISTA_OFF_SCREEN'] = 'True'
os.environ['PYVISTA_USE_PANEL'] = 'True'

DIR = os.path.dirname(os.getcwd())
sys.path.append(DIR)
from torch_points3d.datasets.segmentation.scannet import SCANNET_COLOR_MAP

In [ ]:
VIZ_REL_DIR = "outputs/2020-07-24/10-55-05/eval/2020-07-28_14-59-26/viz"
VIZ_DIR = os.path.join(DIR, VIZ_REL_DIR)
all_viz_data = glob.glob(os.path.join(VIZ_DIR,'*.pt'))

In [ ]:
file_idx = 2
sample_idx = 0
data = torch.load(all_viz_data[file_idx])
data

In [ ]:
data.pred_boxes[0][0].classname

In [ ]:
data = torch.load(all_viz_data[file_idx])
confidence_threshold = 0.7
def load_random_sample(event):
    i = np.random.randint(0, data.pos.shape[0])
     
    pl1 = pv.Plotter(notebook=True)
    pl2 = pv.Plotter(notebook=True)
    
    # Color by points with a label
    pl1.add_points(data.pos[i].numpy(), color="gray", opacity=0.4) 
    pl2.add_points(data.pos[i].numpy(), color="gray", opacity=0.4) 
    
    gt_boxes = data.gt_boxes[i]
    for k in range(len(gt_boxes)):
        xmin, ymin, zmin = gt_boxes[k].corners3d.min(0)
        xmax, ymax, zmax = gt_boxes[k].corners3d.max(0)
        box = pv.Box([xmin,xmax,ymin,ymax,zmin,zmax])
        color = np.asarray(SCANNET_COLOR_MAP[gt_boxes[k].classname]) / 255.
        pl1.add_mesh(box, color=color, show_edges=True, opacity=0.5)
        
    pred_boxes = data.pred_boxes[i]
    for k in range(len(pred_boxes)):
        if pred_boxes[k].score < confidence_threshold: continue
        xmin, ymin, zmin = pred_boxes[k].corners3d.min(0)
        xmax, ymax, zmax = pred_boxes[k].corners3d.max(0)
        box = pv.Box([xmin,xmax,ymin,ymax,zmin,zmax])
        color = np.asarray(SCANNET_COLOR_MAP[pred_boxes[k].classname]) / 255.
        pl2.add_mesh(box, color=color, show_edges=True, opacity=0.5)

    pan1.object = pl1.ren_win
    pan2.object = pl2.ren_win

In [ ]:
pl1 = pv.Plotter(notebook=True)
pl2 = pv.Plotter(notebook=True)
pan1 = pn.panel(pl1.ren_win, sizing_mode='scale_both', aspect_ratio=1,orientation_widget=False,)
pan2 = pn.panel(pl2.ren_win, sizing_mode='scale_both', aspect_ratio=1,orientation_widget=False,)
button = pn.widgets.Button(name='Load new model', button_type='primary')
button.on_click(load_random_sample)
pn.Row(
    pn.Column('## Votenet visualizer',button,'Threshold = 0.7', width=200),
    pn.Column(pan1,'Ground truth'),pn.Column(pan2, 'Box Predictions'),
)